#### 1. Baseline: 41 % Accuracy, 0.40 F1 Score
#### 2. Parameter Raster bauen, alle Kombinationen per Grid Search durchlaufen
#### 3. Metriken, inkl. Accuracy, F1, Overfitting Indikatoren und Trainingszeit
#### 4. Analyse mit Heatmaps, etc.

In [ ]:
import itertools
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score
import xgboost as xgb

learning rate --- max_depth

Tiefere Bäume benötigen niedrige Lernraten um Overfitting zu vermeiden und stabile Konvergenz zu gewährleisten. Hohe Lernraten mit tiefen Bäumen führen zu instabilem Training.

In [ ]:
learning_rates = [0.01, 0.05, 0.1, 0.2]
max_depths = [3, 5, 7, 9, 12]

results = []

In [ ]:
for lr, md in itertools.product(learning_rates, max_depths):
    scores = []
   
    model = xgb.XGBClassifier(
    objective="multi:softprob",
    num_class= 5,
    eval_metric="mlogloss",
    use_label_encoder=False,
    random_state= 42,
    learning_rate=lr,
    max_depth=md,
    n_estimators=200  # fix, wenn nicht die zweite Hypothese
        )